## Fake News Detection: CNN

In [ ]:
# imports
import pandas as pd
import os
import csv
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import utils

ModuleNotFoundError: No module named 'utils'

In [ ]:
os.chdir("/content")
print(os.getcwd())
os.listdir()

/content


['.config', 'WELFake_Dataset.csv', 'sample_data']

In [ ]:
data = pd.read_csv("WELFake_Dataset.csv", on_bad_lines='skip', delimiter=',', quotechar='"', encoding='utf-8')
data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


### 1. Analyze Titles

### 1.1 Remove NA Titles

In [ ]:
data = data.dropna(subset=['title'])
data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


#### 1.2 Split Data

In [ ]:
X = data['title']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


### 1.3 Create CNN

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, nclasses, window_size, embedding_dim, filter_multiplier):
      super(TextCNN, self).__init__()
      self.conv = nn.Conv2d(in_channels = 1, out_channels= filter_multiplier, stride = CONV_STRIDE, kernel_size = 3, padding =1)
      self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = CONV_STRIDE, padding = 1)
      self.linear = nn.Linear(4800, out_features = nclasses)
      self.softmax = nn.LogSoftmax(1)

    def forward(self, x, **kwargs):
      x = torch.transpose(x, 1, 2)
      x = torch.unsqueeze(x, 1)
      x = self.conv(x)
      x = self.maxpool(x)
      F.relu(x)
      x = x.flatten(start_dim = 1)

      x = self.linear(x)
      F.relu(x)
      x = self.softmax(x)
      return x


#### 1.4 Train Model

In [ ]:
num_classes = 12
batch_size = 32
learning_rate = .25
num_epochs = 10

model = TextCNN(num_classes, 3, 300, 100)


optimizer = optim.SGD(model.parameters(), lr=learning_rate, momemntum = 0.8)
lossfn = nn.NLLLoss()

for epoch in range(num_epochs):
  running_loss = 0
  for i, batch in enumerate(utils.batch_iter(data, labels, batch_size = batch_size, num_epochs = 1)):
    input_ = batch[0]
    label = batch[1]
    optimizer.zero_grad()
    output = model(input_)
    loss = lossfn(output, label)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if True:
      print('[%d, %5d] loss: %.6f' % (epoch + 1, i + 1, running_loss / batch_size))
      running_loss = 0

  model = model.eval()
  num_correct = 0
  num_examples = 0


for i, batch in enumerate(utils.batch_iter(test_data, test_labels, batch_size=batch_size, num_epochs=1)):
  y, target = batch
  output = model(y)
  y_pred = []
  # 32x12 -> need to convert softmax to actual class number
  for o in output:
     y_pred.append(torch.max(o, -1).indices.item())
     print("batch ", i, ": ", y_pred, target)
     y_pred = torch.FloatTensor(y_pred)
     correct = torch.eq(torch.round(y_pred).type(target.type()), target).view(-1)
     num_correct += torch.sum(correct).item()
     num_examples += correct.shape[0]
print("Accuracy: ",  num_correct / num_examples)
